# Libraries

In [ ]:
import os
import easydict
import time

import numpy as np
import cv2
import math

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim

from torch.utils import data as D
from torch.utils.data import DataLoader

from tqdm import tqdm
import glob

# Mount google drive to colab environment
The location will at /content/gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Check mounted google drive and its contens
Make sure you to save data inside google driver

In [ ]:
!ls -l
!pwd

total 8
drwx------ 4 root root 4096 Dec 21 05:15 gdrive
drwxr-xr-x 1 root root 4096 Dec 18 16:52 sample_data
/content


In [ ]:
!ls -l "/content/gdrive/My Drive/div2k_100"

total 12
drwx------ 2 root root 4096 Dec 19 18:19 test_images
drwx------ 2 root root 4096 Dec 19 18:19 train_patches_x4lr64
drwx------ 2 root root 4096 Dec 19 18:19 valid_patches_x4lr64


In [ ]:
#!unzip -q /content/div2k_100.zip -d div2k_100

In [ ]:
!python --version
print(torch.__version__)
print(torch.cuda.is_available())

Python 3.6.9
1.3.1
True


# Set your data directory into div2k_dir

In [ ]:
div2k_dir = "/content/gdrive/My Drive/div2k_100"

# Options to control overal programming behaviors

In [ ]:
opt = easydict.EasyDict({
    "resume": False,
    "resume_best": False,
    "use_npy": True,
    
    "multi_gpu": True,
    "use_cuda": True,
    "device": 'cpu',

    "n_epochs": 100, # Total number of epoch to iterate
    "batch_size": 50, # Size of batch of one epoch
    "start_epoch": 1,
    "lr": 1e-4, # Adam: learning rate
    "b1": 0.9, # Adam: decay of first order momentum of gradient
    "b2": 0.999, # Adam: decay of second order momentum of gradient
    
    "checkpoint_dir": None,
    "data_dir": None,
    "train_dir": None,
    "valid_dir": None,
    "test_dir": None,
    "test_result_dir": None,
    
    "lr_img": 64,
    "res_scale": 4,
    "n_channels": 3
})

opt.data_dir = div2k_dir
opt.checkpoint_dir = os.path.join(opt.data_dir, "checkpoint_dir")
opt.train_dir = os.path.join(opt.data_dir, "train_patches_x" + str(opt.res_scale) + "lr" + str(opt.lr_img))
opt.valid_dir = os.path.join(opt.data_dir, "valid_patches_x" + str(opt.res_scale) + "lr" + str(opt.lr_img))
opt.test_dir = os.path.join(opt.data_dir, "test_images")
opt.test_result_dir = os.path.join(opt.data_dir, "test_result")

print(opt)

{'resume': False, 'resume_best': False, 'use_npy': True, 'multi_gpu': True, 'use_cuda': True, 'device': 'cpu', 'n_epochs': 100, 'batch_size': 50, 'start_epoch': 1, 'lr': 0.0001, 'b1': 0.9, 'b2': 0.999, 'checkpoint_dir': '/content/gdrive/My Drive/div2k_100/checkpoint_dir', 'data_dir': '/content/gdrive/My Drive/div2k_100', 'train_dir': '/content/gdrive/My Drive/div2k_100/train_patches_x4lr64', 'valid_dir': '/content/gdrive/My Drive/div2k_100/valid_patches_x4lr64', 'test_dir': '/content/gdrive/My Drive/div2k_100/test_images', 'test_result_dir': '/content/gdrive/My Drive/div2k_100/test_result', 'lr_img': 64, 'res_scale': 4, 'n_channels': 3}


# Noramlization of image
This function will make image range from 0 ~ 255 to 0 ~ 1.0 and convert type from int16 to float32

In [ ]:
def normalize_img(img):
    img = img / 255.
    img = img.astype(np.float32)
    return img

# Define your dataset
This defined dataset can be easily loaded by pytorch library

In [ ]:
class DatasetFromFolder(data.Dataset):
    def __init__(self, data_dir, use_npy):
        super(DatasetFromFolder, self).__init__()

        self.use_npy = use_npy

        if self.use_npy:
            lr_dir = os.path.join(data_dir, 'lr.npy')
            hr_dir = os.path.join(data_dir, 'hr.npy')
        else:
            lr_dir = os.path.join(data_dir, 'lr')
            hr_dir = os.path.join(data_dir, 'hr')
        
        self.dsets = {}

        if self.use_npy:
            self.dsets['lr'] = np.load(lr_dir)
            self.dsets['hr'] = np.load(hr_dir)
        else:
            lr_list = os.listdir(lr_dir)
            hr_list = os.listdir(hr_dir)

            lr_list.sort()
            hr_list.sort()
            self.dsets['lr'] = [os.path.join(lr_dir, x) for x in lr_list]
            self.dsets['hr'] = [os.path.join(hr_dir, x) for x in hr_list]
        
        self.dsets['file_name'] = os.listdir(os.path.join(data_dir, 'hr'))

    def __getitem__(self, idx):

        if self.use_npy:
            input = self.dsets['lr'][idx]
            target = self.dsets['hr'][idx]
        else:
            input = cv2.imread(self.dsets['lr'][idx])
            target = cv2.imread(self.dsets['hr'][idx])

        file_name = self.dsets['file_name'][idx]

        input = cv2.resize(input, (target.shape[1], target.shape[0]), interpolation=cv2.INTER_CUBIC)
        
        input = normalize_img(input)
        target = normalize_img(target)

        input = np.transpose(input, (2, 0, 1))
        target = np.transpose(target, (2, 0, 1))

        input = torch.from_numpy(input).type(torch.FloatTensor)
        target = torch.from_numpy(target).type(torch.FloatTensor)

        return input, target, file_name

    def __len__(self):
        return len(self.dsets['lr'])

# Shift function
This function will shift your image from 0 ~ 1.0 to -mean ~ (1 - mean) for each RGB channel. This will make neural network to learn easily

In [ ]:
class MeanShift(nn.Conv2d):
    def __init__(
        self, rgb_range,
        rgb_mean=(0.4488, 0.4371, 0.4040), rgb_std=(1.0, 1.0, 1.0), sign=-1):

        super(MeanShift, self).__init__(3, 3, kernel_size=1)
        std = torch.Tensor(rgb_std)
        self.weight.data = torch.eye(3).view(3, 3, 1, 1) / std.view(3, 1, 1, 1)
        self.bias.data = sign * rgb_range * torch.Tensor(rgb_mean) / std
        for p in self.parameters():
            p.requires_grad = False

# Define your model
See details of [paper](https://arxiv.org/abs/1501.00092)

In [ ]:
class SRCNN(nn.Module):
    def __init__(self, opt):
        super(SRCNN, self).__init__()
        pix_range = 1.0
        
        self.sub_mean = MeanShift(pix_range)
        self.add_mean = MeanShift(pix_range, sign=1)
        
        self.conv1 = nn.Conv2d(opt.n_channels, 64, kernel_size=15, padding=7)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, opt.n_channels, kernel_size=5, padding=2)

    def forward(self, x):
        x = self.sub_mean(x)
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.conv3(x)
        
        out = self.add_mean(x)
        return out

# Prepare for your unwanted system or program down
This function saves the progress of learning for each echo. You can load the mode later from the last learned model. Of course, you have to define the function of loading model.

In [ ]:
def save_checkpoint(srcnn, epoch, loss):
    checkpoint_dir = os.path.abspath(opt.checkpoint_dir)
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    checkpoint_path = os.path.join(checkpoint_dir, "models_epoch_%04d_loss_%.6f.pth" % (epoch, loss))
    
    if torch.cuda.device_count() > 1 and opt.multi_gpu:
        state = {"epoch": epoch, "srcnn": srcnn.module}
    else:
        state = {"epoch": epoch, "srcnn": srcnn}

    torch.save(state, checkpoint_path)
    print("Checkpoint saved to {}".format(checkpoint_path))

In [ ]:
def load_model(checkpoint_dir):
    checkpoint_list = glob.glob(os.path.join(checkpoint_dir, "*.pth"))
    checkpoint_list.sort()

    if opt.resume_best:
        loss_list = list(map(lambda x: float(os.path.basename(x).split('_')[4][:-4]), checkpoint_list))
        best_loss_idx = loss_list.index(min(loss_list))
        checkpoint_path = checkpoint_list[best_loss_idx]
    else:
        checkpoint_path = checkpoint_list[len(checkpoint_list) - 1]

    srcnn = SRCNN(opt)

    if os.path.isfile(checkpoint_path):
        print("=> loading checkpoint '{}'".format(checkpoint_path))
        checkpoint = torch.load(checkpoint_path)
        
        n_epoch = checkpoint['epoch']
        srcnn.load_state_dict(checkpoint['srcnn'].state_dict())
        print("=> loaded checkpoint '{}' (epoch {})"
                .format(checkpoint_path, n_epoch))
    else:
        print("=> no checkpoint found at '{}'".format(checkpoint_path))
        n_epoch = 0

    return n_epoch + 1, srcnn

# Define your train step

In [ ]:
def train(opt, model, optimizer, data_loader, loss_criterion):
    print("===> Training")
    start_time = time.time()

    total_loss = 0.0
    total_psnr = 0.0

    for iteration, batch in enumerate(tqdm(data_loader), 1):
        x, target = batch[0], batch[1]
        if opt.use_cuda:
            x = x.to(opt.device)
            target = target.to(opt.device)

        out = model(x)

        loss = loss_criterion(out, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        psnr = 10 * math.log10(1. / loss.item())
        total_psnr += psnr

        # print("Training %.2fs => Epoch[%d/%d](%d/%d): Loss: %.5f PSNR: %.5f" %
        #     (time.time() - start_time, opt.epoch_num, opt.n_epochs, iteration, len(data_loader), loss.item(), psnr))

    total_loss = total_loss / iteration
    total_psnr = total_psnr / iteration
    
    print("***Training %.2fs => Epoch[%d/%d]: Loss: %.5f PSNR: %.5f" %
        (time.time() - start_time, opt.epoch_num, opt.n_epochs, total_loss, total_psnr))

    return (total_loss, total_psnr)

# Define your evaluation method for validation dataset

In [ ]:
def evaluate(opt, model, data_loader, loss_criterion):
    print("===> Validation")
    start_time = time.time()

    total_loss = 0.0
    total_psnr = 0.0
    with torch.no_grad():
        for iteration, batch in enumerate(tqdm(data_loader), 1):
            x, target = batch[0], batch[1]

            if opt.use_cuda:
                x = x.to(opt.device)
                target = target.to(opt.device)

            out = model(x)
            
            loss = loss_criterion(out, target)
            total_loss += loss.item()

            psnr = 10 * math.log10(1. / loss.item())
            total_psnr += psnr

            # print("Validation %.2fs => Epoch[%d/%d](%d/%d): Loss: %.5f PSNR: %.5f" %
            #     (time.time() - start_time, opt.epoch_num, opt.n_epochs, iteration, len(data_loader), loss.item(), psnr))

    total_loss = total_loss / iteration
    total_psnr = total_psnr / iteration
    
    print("***Validation %.2fs => Epoch[%d/%d]: Loss: %.5f PSNR: %.5f" %
        (time.time() - start_time, opt.epoch_num, opt.n_epochs, total_loss, total_psnr))

    return (total_loss, total_psnr)

# Define whole training process

In [ ]:
def run_train(opt, training_data_loader, validation_data_loader):
    # Define what device we are using
    if not os.path.exists(opt.checkpoint_dir):
        os.makedirs(opt.checkpoint_dir)

    log_file = os.path.join(opt.checkpoint_dir, "srcnn_log.csv")

    print('[Initialize networks for training]')
    srcnn = SRCNN(opt)
    l2_criterion = nn.MSELoss()
    print(srcnn)

    # optionally resume from a checkpoint
    if opt.resume:
        opt.start_epoch, srcnn = load_model(opt.checkpoint_dir)
    else:
        with open(log_file, mode='w') as f:
            f.write("epoch,train_loss,valid_loss\n")

    print("===> Setting GPU")
    print("CUDA Available: ", torch.cuda.is_available())
    if opt.use_cuda and torch.cuda.is_available():
        opt.use_cuda = True
        opt.device = 'cuda'
    else:
        opt.use_cuda = False
        opt.device = 'cpu'
        
    if torch.cuda.device_count() > 1 and opt.multi_gpu:
        print("Use " + str(torch.cuda.device_count()) + " GPUs")
        srcnn = nn.DataParallel(srcnn)

    if opt.use_cuda:
        srcnn = srcnn.to(opt.device)
        l2_criterion = l2_criterion.to(opt.device)

    print("===> Setting Optimizer")
    optimizer = torch.optim.Adam(srcnn.parameters(),  lr=opt.lr, betas=(opt.b1, opt.b2))

    for epoch in range(opt.start_epoch, opt.n_epochs):
        opt.epoch_num = epoch
        train_loss, train_psnr = train(opt, srcnn, optimizer, training_data_loader, loss_criterion=l2_criterion)
        valid_loss, valid_psnr = evaluate(opt, srcnn, validation_data_loader, loss_criterion=l2_criterion)

        with open(log_file, mode='a') as f:
            f.write("%d,%08f,%08f,%08f,%08f\n" % (
                epoch,
                train_loss,
                train_psnr,
                valid_loss,
                valid_psnr
            ))
        save_checkpoint(srcnn, epoch, valid_loss)

# Now it's time to train the SRCNN model

In [ ]:
train_dir = opt.train_dir
valid_dir = opt.valid_dir
print("train_dir is: {}".format(train_dir))
print("valid_dir is: {}".format(valid_dir))

target_size = (opt.lr_img * opt.res_scale, opt.lr_img * opt.res_scale)


train_dataset = DatasetFromFolder(train_dir, opt.use_npy)
valid_dataset = DatasetFromFolder(valid_dir, opt.use_npy)

training_data_loader = DataLoader(dataset=train_dataset,
                                  batch_size=opt.batch_size,
                                  shuffle=True)
validation_data_loader = DataLoader(dataset=valid_dataset,
                                    batch_size=opt.batch_size,
                                    shuffle=False)

run_train(opt, training_data_loader, validation_data_loader)

train_dir is: /content/gdrive/My Drive/div2k_100/train_patches_x4lr64
valid_dir is: /content/gdrive/My Drive/div2k_100/valid_patches_x4lr64
[Initialize networks for training]
SRCNN(
  (sub_mean): MeanShift(3, 3, kernel_size=(1, 1), stride=(1, 1))
  (add_mean): MeanShift(3, 3, kernel_size=(1, 1), stride=(1, 1))
  (conv1): Conv2d(3, 64, kernel_size=(15, 15), stride=(1, 1), padding=(7, 7))
  (relu1): ReLU()
  (conv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(32, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
)
===> Setting GPU
CUDA Available:  True


  0%|          | 0/71 [00:00<?, ?it/s]

===> Setting Optimizer
===> Training


  0%|          | 0/11 [00:00<?, ?it/s]

***Training 28.39s => Epoch[1/100]: Loss: 0.02385 PSNR: 17.26001
===> Validation


...

***Validation 2.70s => Epoch[98/100]: Loss: 0.00210 PSNR: 27.69055
Checkpoint saved to /content/gdrive/My Drive/div2k_100/checkpoint_dir/models_epoch_0098_loss_0.002099.pth
===> Training


  0%|          | 0/11 [00:00<?, ?it/s]

***Training 28.14s => Epoch[99/100]: Loss: 0.00276 PSNR: 25.67591
===> Validation


100%|██████████| 11/11 [00:02<00:00,  4.25it/s]

***Validation 2.71s => Epoch[99/100]: Loss: 0.00211 PSNR: 27.66062
Checkpoint saved to /content/gdrive/My Drive/div2k_100/checkpoint_dir/models_epoch_0099_loss_0.002109.pth


# Test model
We will use PSNR and SSIM metrics to measure how well the model is trained. We can skimage libraries to easily measure two metrics.

In [ ]:
from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_psnr as psnr

# Define test dataset
This time, we will not use dataloader of pytorch function. Instead, we will load each image from test data directory.

In [ ]:
def get_test_dataset(data_dir, res_scale):
    lr_dir = os.path.join(data_dir, 'lrx' + str(res_scale))
    hr_dir = os.path.join(data_dir, 'hr')
        
    dsets = {}
    lr_list = os.listdir(lr_dir)
    hr_list = os.listdir(hr_dir)
    lr_list.sort()
    hr_list.sort()
    dsets['lr'] = [os.path.join(lr_dir, x) for x in lr_list]
    dsets['hr'] = [os.path.join(hr_dir, x) for x in hr_list]
    dsets['file_name'] = os.listdir(hr_dir)
    
    return dsets

# Define your test method of the model

In [ ]:
def test_model(opt, test_dataset):
    
    lr_list = test_dataset['lr']
    hr_list = test_dataset['hr']
    filename_list = test_dataset['file_name']
    
    sr_compare_dir = os.path.join(opt.test_result_dir, "compare")
    sr_result_dir = os.path.join(opt.test_result_dir, "sr")

    if not os.path.exists(sr_result_dir):
        os.makedirs(sr_result_dir)
    if not os.path.exists(sr_compare_dir):
        os.makedirs(sr_compare_dir)

    opt.resume_best = True
    _, srcnn = load_model(opt.checkpoint_dir)
    criterion = nn.MSELoss()

    if torch.cuda.device_count() > 1 and opt.multi_gpu:
        print("Use " + str(torch.cuda.device_count()) + " GPUs")
        srcnn = nn.DataParallel(srcnn)

    if opt.use_cuda and torch.cuda.is_available():
        opt.use_cuda = True
        opt.device = 'cuda'
    else:
        opt.use_cuda = False
        opt.device = 'cpu'

    if opt.use_cuda:
        srcnn = srcnn.to(opt.device)
        criterion = criterion.to(opt.device)

    hr_img_sz = (opt.lr_img * opt.res_scale, opt.lr_img * opt.res_scale)
    result_img = np.zeros((hr_img_sz[0], hr_img_sz[1] * 3, opt.n_channels))

    with torch.no_grad():
        total_num = 0
        sum_bicubic_psnr = 0.
        sum_sr_psnr = 0.
        sum_bicubic_ssim = 0.
        sum_sr_ssim = 0.
        
        avg_bicubic_psnr = 0.
        avg_sr_psnr = 0.
        avg_bicubic_ssim = 0.
        avg_sr_ssim = 0.

        start_time = time.time()
        for batch in zip(lr_list, hr_list, filename_list):
            input_path, target_path, file_name = batch[0], batch[1], batch[2]

            # print(input_path)
            # print(target_path)
            
            input = cv2.imread(input_path)
            target = cv2.imread(target_path)
            
            input = cv2.resize(input, (target.shape[1], target.shape[0]), interpolation=cv2.INTER_CUBIC)
            
            input = normalize_img(input)
            target = normalize_img(target)
            
            input = np.transpose(input, (2, 0, 1))
            target = np.transpose(target, (2, 0, 1))
            
            input = input.reshape(1, input.shape[0], input.shape[1], input.shape[2])
            target = target.reshape(1, target.shape[0], target.shape[1], target.shape[2])
            
            input = torch.from_numpy(input).type(torch.FloatTensor)
            target = torch.from_numpy(target).type(torch.FloatTensor)

            if opt.use_cuda:
                input = input.to(opt.device)
#                 target = target.to(opt.device)

            out = srcnn(input)
            
            for i in range(input.size(0)): 
                if opt.use_cuda:
                    input_arr = input[i].detach().to('cpu').data.numpy()
                    sr_arr = out[i].detach().to('cpu').data.numpy()
                else:
                    input_arr = input[i].data.numpy()
                    sr_arr = out[i].data.numpy()
                
                target_arr = target[i].detach().data.numpy()
        
                input_arr = np.transpose(input_arr, (1, 2, 0))
                sr_arr = np.transpose(sr_arr, (1, 2, 0))
                target_arr = np.transpose(target_arr, (1, 2, 0))
                
                sr_arr[sr_arr < 0.] = 0
                sr_arr[sr_arr > 1.] = 1.
                
                compare_img = np.concatenate((input_arr, sr_arr, target_arr), axis=1)
            
                bicubic_psnr = psnr(input_arr, target_arr)
                sr_psnr = psnr(sr_arr, target_arr)
                
                sum_bicubic_psnr += bicubic_psnr
                sum_sr_psnr += sr_psnr
                
                bicubic_ssim = ssim(input_arr, target_arr, multichannel=True)
                sr_ssim = ssim(sr_arr, target_arr, multichannel=True)
                
                sum_bicubic_ssim += bicubic_ssim
                sum_sr_ssim += sr_ssim
                
                
                
                compare_img = compare_img * 255
                compare_img = compare_img.astype(np.int16)
                sr_arr = sr_arr * 255
                sr_arr = sr_arr.astype(np.int16)
                cv2.imwrite(os.path.join(sr_compare_dir, file_name), compare_img)
                cv2.imwrite(os.path.join(sr_result_dir, file_name), sr_arr)
                
                print(file_name)
                print("Bicubic PSNR: {:.8f}, Bicubic SSIM: {:.8f}, SR PSNR: {:.8f}, SR SSIM: {:.8f}".format(
                    bicubic_psnr, bicubic_ssim, sr_psnr, sr_ssim))
                total_num += 1

        avg_bicubic_psnr = sum_bicubic_psnr / total_num
        avg_sr_psnr = sum_sr_psnr / total_num
        
        avg_bicubic_ssim = sum_bicubic_ssim / total_num
        avg_sr_ssim = sum_sr_ssim / total_num

        print("Time: {:.2f}".format(time.time() - start_time))
        print("Bicubic PSNR: {:.8f}".format(avg_bicubic_psnr))
        print("Bicubic SSIM: {:.8f}".format(avg_bicubic_ssim))
        print("SR PSNR: {:.8f}".format(avg_sr_psnr))
        print("SR SSIM: {:.8f}".format(avg_sr_ssim))

In [ ]:
test_dir = opt.test_dir
res_scale = opt.res_scale
print("test_dir is: {}".format(test_dir))
print("test_result_dir is: {}".format(opt.test_result_dir))

test_dataset = get_test_dataset(test_dir, res_scale)

test_model(opt, test_dataset)

test_dir is: /content/gdrive/My Drive/div2k_100/test_images
test_result_dir is: /content/gdrive/My Drive/div2k_100/test_result
=> loading checkpoint '/content/gdrive/My Drive/div2k_100/checkpoint_dir/models_epoch_0095_loss_0.002098.pth'
=> loaded checkpoint '/content/gdrive/My Drive/div2k_100/checkpoint_dir/models_epoch_0095_loss_0.002098.pth' (epoch 95)
0830.png
Bicubic PSNR: 23.65051715, Bicubic SSIM: 0.77931095, SR PSNR: 24.02120494, SR SSIM: 0.79954338
0883.png
Bicubic PSNR: 25.04664727, Bicubic SSIM: 0.79283191, SR PSNR: 25.56328485, SR SSIM: 0.82098483
0884.png
Bicubic PSNR: 24.20699310, Bicubic SSIM: 0.77717060, SR PSNR: 24.67301781, SR SSIM: 0.79965764
0886.png
Bicubic PSNR: 33.58014167, Bicubic SSIM: 0.96164323, SR PSNR: 34.37567642, SR SSIM: 0.96379398
0891.png
Bicubic PSNR: 23.90350388, Bicubic SSIM: 0.81219866, SR PSNR: 24.56720817, SR SSIM: 0.83302189
Time: 28.84
Bicubic PSNR: 26.07756061
Bicubic SSIM: 0.82463107
SR PSNR: 26.64007844
SR SSIM: 0.84340034
